In [1]:
import numpy as np
import cv2
import os
import pandas as pd

In [2]:
from sewar.full_ref import mse, rmse, psnr, rmse_sw, uqi, ssim, ergas, scc, rase, sam, msssim, vifp, psnrb 
metrics = {'mse':mse, 'rmse':rmse, 'psnr':psnr, 'rmse_sw':rmse_sw, 'uqi':uqi, 'ssim':ssim, 
    'ergas':ergas, 'scc':scc, 'rase':rase, 'sam': sam, 'msssim': msssim, 'vifp':vifp, 'psnrb':psnrb }

Extract images from video in Python

https://www.geeksforgeeks.org/extract-images-from-video-in-python/

OpenCV - Getting Started with Videos

https://docs.opencv.org/3.4/dd/d43/tutorial_py_video_display.html

In [3]:
def save_image(video_name, frame, slide, num_frame):
    name = f'./data/{video_name}/frame{slide}-{num_frame}.jpg'
    cv2.imwrite(name, frame)

# Saving images with thresholds

In [6]:
videos_name = [
    # 'Introduction',
    'Training at scale with Vertex AI',
    # 'Lab intro_ Training at scale with the Vertex AI Training Service'
]

MSE_SAME = 0.07 # 0.07 - 0.1
SAMPLING_RATE = 0.1 # 0.05 - 0.14 seconds
SAMPLING_CHK_TIME = 0.6

MSE_NEW = 4
MSE_VALID = 15.53
MSSSIM_SAME = 0.998038
# MSSSIM_SAME = 0.997781
WHITE_FRAME = 528757248
BLACK_FRAME = 2339

def is_static(val):
    return val < MSE_SAME


def check_bw_frame(frame, idx):
    bw_check = np.sum(frame)
    valid_frame = True

    if(bw_check >= WHITE_FRAME):
        print(f'X Image{idx} -> white')
        valid_frame = False
    elif(bw_check <= BLACK_FRAME):
        print(f'X Image{idx} -> black')
        valid_frame = False

    return valid_frame, bw_check


for video_name in videos_name:

    print(f'\n{video_name}\n')

    if not os.path.exists(f'data/{video_name}'):
        os.makedirs(f'data/{video_name}')
    
    video_path = f'/mnt/c/Users/Eduardo/Downloads/{video_name}.mp4'
    cam = cv2.VideoCapture(video_path)

    currentframe = 0
    num_slide = 1

    #### Check if video start static
    _, frame = cam.read()
    currentframe += 1
    last_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    last_image_gray = last_frame_gray

    static_found = False

    frame_step = round(SAMPLING_RATE * cam.get(cv2.CAP_PROP_FPS))
    num_frames_chk = round(SAMPLING_CHK_TIME * cam.get(cv2.CAP_PROP_FPS))

    while((static_found == False) and (currentframe < num_frames_chk)):
        for  _ in range(frame_step):
            ret, frame = cam.read()
            currentframe += 1

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        similarity = mse(frame_gray, last_frame_gray)

        if(is_static(similarity)):
            valid_image, bw = check_bw_frame(last_image_gray, currentframe)

            if(valid_image):
                print(f'Image{currentframe} -> {bw}')
                save_image(video_name, frame, num_slide, currentframe)
                num_slide += 1

            static_found = True

        last_frame_gray = frame_gray
        last_image_gray = frame_gray
        
    #### end

    while(True):
        
        for  _ in range(frame_step):
            ret, frame = cam.read()
            currentframe += 1
    
        if ret:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            similarity = mse(last_frame_gray, frame_gray)

            if(is_static(similarity)):
                newimg_thresh = mse(frame_gray, last_image_gray)

                if(newimg_thresh >= MSE_NEW):
                    valid_image = True

                    if(newimg_thresh <= MSE_VALID):
                        double_check = msssim(frame_gray, last_image_gray).real
                        log_info = f'Image{num_slide}-{currentframe}-> {newimg_thresh:.2f}, {double_check:5f} msssim'
                        if(double_check > MSSSIM_SAME):
                            print('X ' + log_info)
                            valid_image = False
                        else:
                            print(f'F ' + log_info)

                    if(valid_image):
                        valid_image, bw = check_bw_frame(frame_gray, currentframe)

                    if(valid_image):
                        print(f'Image{num_slide}-{currentframe} -> {newimg_thresh:.2f}, {bw}')
                        save_image(video_name, frame, num_slide, currentframe)
                        num_slide += 1
                    
                    last_image_gray = frame_gray
                        
            last_frame_gray = frame_gray
        else:
            break

    cam.release()


Training at scale with Vertex AI

Image1-31 -> 1290.69, 488118736
X Image2-166-> 5.34, 0.999331 msssim
Image2-751 -> 3025.34, 520781512
Image3-1252 -> 2632.94, 496933249
X Image4-1288-> 4.68, 0.999251 msssim
X Image4-1447-> 6.06, 0.999042 msssim
X Image4-1606-> 5.06, 0.999128 msssim
X Image4-1768-> 4.65, 0.999138 msssim
X Image4-1927-> 4.55, 0.999438 msssim
Image4-2209 -> 2698.34, 515836264
Image5-2692 -> 1870.74, 510891660
Image6-2884 -> 1479.46, 524411925
Image7-2968 -> 95.08, 523079611
Image8-3049 -> 104.22, 521614524
Image9-3160 -> 121.86, 519915275
Image10-3295 -> 1455.66, 517955133
Image11-3565 -> 73.66, 516497988
Image12-3670 -> 1373.73, 517577173
Image13-3799 -> 2570.63, 499083797
X Image14-3847-> 5.11, 0.999356 msssim
X Image14-4006-> 4.54, 0.999532 msssim
X Image14-4486-> 4.28, 0.999558 msssim
X Image14-5125-> 4.23, 0.999552 msssim
X Image14-5605-> 4.26, 0.999575 msssim
Image14-5953 -> 2816.76, 501426177
X Image15-6088-> 6.61, 0.999456 msssim
Image15-6406 -> 2132.64, 5085620

## Compare Metrics

* Video: What kinds of problems can it solve_.mp4
* Frame: 4500 (same)
    * mse: 5.704356	
    * msssim: 0.999670
* Frame: 8358 (little diff)
    * mse: 4.548090
    * msssim: 0.997047
* Frame: 8591 (little diff)
    * 4.0318262924382715

Results research
* same_image -> msssim = 1, msssim >= 0.999670
* min_change -> 0.997047

* Infuse your apps with ML
* Image3-403 -> 0.998119 msssim (same)

In [ ]:
video_name = "What kinds of problems can it solve_.mp4"
imgs_idx = [0, 167, 511, 1309, 1383, 1590, 1791, 1896, 2791, 3502, 4466, 4500, 5045, 5978, 6754, 7397, 8035, 8358, 8668, 8866]
# test_metrics = ['mse', 'rmse', 'psnr', 'uqi', 'ergas', 'scc', 'rase', 'sam', 'msssim', 'vifp', 'psnrb']
test_metrics = ['ssim']

cam = cv2.VideoCapture(video_name)
_, last_frame = cam.read()
last_image = last_frame
currentframe = 1

data = {'img': imgs_idx[1:]}

for metric in test_metrics:
    data[metric] = []

while(True):
    ret, frame = cam.read()
  
    if ret:
        if(currentframe in imgs_idx):
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            last_image_gray = cv2.cvtColor(last_image, cv2.COLOR_BGR2GRAY)

            for metric in test_metrics:
                if metric == 'msssim':
                    data[metric].append(metrics[metric](frame_gray, last_image_gray).real)
                else:
                    data[metric].append(metrics[metric](frame_gray, last_image_gray))
            
            last_image = frame
            
        currentframe += 1
        last_frame = frame
    else:
        break

cam.release()

In [ ]:
df = pd.DataFrame(data)

In [ ]:
video_name = "Course introduction.mp4"
imgs_idx = [0, 139, 267, 314, 459, 585, 713, 794, 938, 1500]

cam = cv2.VideoCapture(video_name)
_, last_frame = cam.read()
last_image = last_frame
currentframe = 1

sim_mse = []
newimg_mse = []
sim_msssim = []
newimg_msssim = []

while(True):
    ret, frame = cam.read()
  
    if ret:
        if(currentframe in imgs_idx):
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            last_frame_gray = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
            last_image_gray = cv2.cvtColor(last_image, cv2.COLOR_BGR2GRAY)

            # sim_mse.append(mse(last_frame_gray, frame_gray))
            # newimg_mse.append(mse(frame_gray, last_image_gray))
            sim_msssim.append(msssim(last_frame_gray, frame_gray).real)
            newimg_msssim.append(msssim(frame_gray, last_image_gray).real)
            
            last_image = frame
            
        currentframe += 1
        last_frame = frame
    else:
        break

data = pd.DataFrame({
    'img': imgs_idx[1:],
    # 'sim_mse': sim_mse,
    # 'newimg_mse': newimg_mse,
    'sim_msssim': sim_msssim,
    'newimg_msssim': newimg_msssim
})

cam.release()
data

## Analyzing two pairs of images

In [ ]:
cam = cv2.VideoCapture(video_name)

imgs_idx = [4438, 4500, 8035, 8358]
imgs = {}
currentframe = 0

while(True):
    ret, frame = cam.read()
  
    if ret:
        if(currentframe in imgs_idx):
            imgs[currentframe] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
        currentframe += 1
    else:
        break

cam.release()

In [ ]:
comp1_2 = []
comp3_4 = []

for metr in metrics:
    comp1_2.append(metr(imgs[imgs_idx[0]], imgs[imgs_idx[1]]))
    comp3_4.append(metr(imgs[imgs_idx[2]], imgs[imgs_idx[3]]))

data = pd.DataFrame({'metric': metrics.keys(), 'comp1_2': comp1_2, 'comp3_4':comp3_4})
data['good'] = data['comp1_2'] > data['comp3_4']
data

## Frames similarity

In [ ]:
cam = cv2.VideoCapture("Course introduction.mp4")
_, last_frame = cam.read()

similarity_all = []

while(True):
    ret, frame = cam.read()
  
    if ret:
        similarity_all.append(mse(last_frame, frame))
        last_frame = frame
    else:
        break

cam.release()